Code implemented by Rosa Mimi Haßfurther, r.hassfurther@tu-berlin.de


In [ ]:
# import packages
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.integrate import solve_ivp

# Diafiltration

In [ ]:
# defining Differential equation for diafiltration
def ode_fcn(t, y, Qp, V0, sigma_a):
    # define parameters
    Cc = y[0]

    # ODE
    dCc_dt = -Qp/V0*(1-sigma_a)*Cc

    return [dCc_dt]

# Solver function
def solver(Cc_0, Qp, V0, sigma_a, nD_end):

    #define the function to solve
    fun = lambda t, y: ode_fcn(t, y, Qp, V0, sigma_a)

    #set initial value for parameters
    y0 = [Cc_0]

    nD_span = np.linspace(0, nD_end, 1000)  # Fine resolution over permeation volume

    #solve the differential equation
    sol = solve_ivp(fun, [nD_span[0], nD_span[-1]], y0, method='LSODA', t_eval=nD_span, rtol=1e-8, atol=1e-8)

    #extract the solution
    nD = sol.t
    Cc = sol.y[0]
    Y = Cc/Cc_0*100
    R = 100*(1-V0*Cc/(V0*Cc_0))

    return nD, Cc, Y, R


In [ ]:
# Set Process Parameters
Cc_0 = 1.0   # Initial concentration Cc at time 0
Qp = 1.0     # permeate flow rate
V0 = 1.0     # initial volume
sigma_a_values = [0.5, 0.7, 0.9, 1.0]  # Different values of sigma_a
nD_end = 10  #  end number permeattion volumes

In [ ]:
# Create figure for Yield (Y)
fig1 = go.Figure()

# Loop over different sigma_a values and plot results
for sigma_a in sigma_a_values:
    nD, Cc, Y, R = solver(Cc_0, Qp, V0, sigma_a, nD_end)

    # Add traces for each sigma_a value
    fig1.add_trace(go.Scatter(x=nD, y=Y, mode='lines', name=f'sigma_a = {sigma_a}'))

# Update layout for Yield plot
fig1.update_layout(title="Evolution of Yield (Y) for Different sigma_a",
                   xaxis_title="nD number of diafiltration volumes",
                   yaxis_title="Yield (%)",
                   width=800)

# Create figure for Solute Removal (R)
fig2 = go.Figure()

# Loop over different sigma_a values and plot results
for sigma_a in sigma_a_values:
    nD, Cc, Y, R = solver(Cc_0, Qp, V0, sigma_a, nD_end)

    # Add traces for each sigma_a value
    fig2.add_trace(go.Scatter(x=nD, y=R, mode='lines', name=f'sigma_a = {sigma_a}'))

# Update layout for Solute Removal plot
fig2.update_layout(title="Evolution of Solute Removal (R) for Different sigma_a",
                   xaxis_title="nD number of diafiltration volumes",
                   yaxis_title="Removal (%)",
                   width=800)

fig1.show()
fig2.show()